Q4 CHATGPT W/O CONSTRAINTS

In [1]:
from gurobipy import Model, GRB
import numpy as np
import pandas as pd

from gurobipy import Model, GRB

# Define the price response function coefficients for the two products
df = pd.read_csv('/Users/pratiksha/Downloads/price_response.csv')  

# Define the price response function coefficients for the products
intercepts = df['Intercept'].to_numpy()
sensitivities = abs(df['Sensitivity'].to_numpy())

# Define the initial prices for the products
initial_prices = np.zeros(len(intercepts))

# Define the step size and stopping criterion for the gradient descent
step_size = 0.001
stopping_criterion = 1e-6

# Define the demand functions for the products
def demand(prices, intercepts, sensitivities):
    return intercepts - sensitivities * prices

# Define the gradient of the objective function
def gradient(prices, intercepts, sensitivities):
    return -2 * sensitivities * (intercepts - sensitivities * prices)

# Initialize the prices
prices = initial_prices

# Create the Gurobi model and variables outside of the loop
m = Model('Projection')
m.setParam('OutputFlag', 0) # Suppress Gurobi output

price_vars = [m.addVar(lb=0, name=f"price_{i}") for i in range(len(prices))]

# Set the initial values for the price variables
for i, price_var in enumerate(price_vars):
    price_var.start = prices[i]

# Add constraints for demand non-negativity
for i, price_var in enumerate(price_vars):
    m.addConstr(intercepts[i] - sensitivities[i] * price_var >= 0, f"DemandNonNegativity_{i}")

# Add constraints for price ordering within each product line
for i in range(3):
    m.addConstr(price_vars[i+3] - price_vars[i] >= 0, f"PriceOrdering_ProductLine_{i}")

# Add constraints for price ordering within the same version
for i in range(3):
    m.addConstr(price_vars[i+6] - price_vars[i+3] >= 0, f"PriceOrdering_Version_{i}")

# Add constraints for price ordering across product lines for the same version
m.addConstr(price_vars[3] - price_vars[0] >= 0.01, name='price_order_basic_1')  # Product 1 Basic must be priced at least $0.01 lower than Product 2 Basic
m.addConstr(price_vars[6] - price_vars[3] >= 0.01, name='price_order_basic_2')  # Product 2 Basic must be priced at least $0.01 lower than Product 3 Basic
m.addConstr(price_vars[4] - price_vars[1] >= 0.01, name='price_order_advanced_1')  # Product 1 Advanced must be priced at least $0.01 lower than Product 2 Advanced
m.addConstr(price_vars[7] - price_vars[4] >= 0.01, name='price_order_advanced_2')  # Product 2 Advanced must be priced at least $0.01 lower than Product 3 Advanced
m.addConstr(price_vars[5] - price_vars[2] >= 0.01, name='price_order_premium_1')  # Product 1 Premium must be priced at least $0.01 lower than Product 2 Premium
m.addConstr(price_vars[8] - price_vars[5] >= 0.01, name='price_order_premium_2')  # Product 2 Premium must be priced at least $0.01 lower than Product 3 Premium

# Start the projected gradient descent algorithm
iteration_counter = 0
while True:
    iteration_counter += 1

    # Compute the gradient at the current prices
    grad = gradient(prices, intercepts, sensitivities)
    
    # Take a step in the direction of the gradient
    new_prices = prices + step_size * grad
    
    # Project the new prices to satisfy the non-negativity constraints
    new_prices = np.maximum(new_prices, np.zeros(len(new_prices)))

    # Update the model with the new objective function
    m.setObjective(sum((price_vars[i] - new_prices[i])*(price_vars[i] - new_prices[i]) for i in range(len(prices))),
                   GRB.MINIMIZE)

    # Optimize the model
    m.optimize()
    
    # Extract the projected prices
    projected_prices = np.array([var.X for var in price_vars])
    
    # Check the stopping criterion
    if np.linalg.norm(prices - projected_prices) < stopping_criterion:
        break
    
    # Update the prices
    prices = projected_prices

# Print the final prices and objective value
print("\n")
print(f'The model ran {iteration_counter} iterations.')
print("Optimal prices found:", prices)
print("Optimal objective value:", m.objVal)


Restricted license - for non-production use only - expires 2025-11-24


The model ran 2 iterations.
Optimal prices found: [0.   0.   0.   0.01 0.01 0.01 0.02 0.02 0.02]
Optimal objective value: 0.0015


/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_33793/2813135144.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


___

Q4 CHATGPT

In [2]:
import pandas as pd
from gurobipy import Model, GRB

# Load the dataset
df = pd.read_csv('/Users/pratiksha/Downloads/price_response.csv') 

# Create a Gurobi model
model = Model('Maximize Revenue Across Line Constraints')

# Convert dataframe columns to arrays
intercepts = df['Intercept'].to_numpy()
sensitivities = abs(df['Sensitivity'].to_numpy())
capacities = df['Capacity'].to_numpy()

# Add price variables for each product
prices = model.addVars(len(df), name='price', lb=0)

# Define the objective function (total revenue)
revenue = sum((intercepts[i] - sensitivities[i] * prices[i]) * prices[i] for i in range(len(df)))
model.setObjective(revenue, GRB.MAXIMIZE)

# Add capacity constraints
for i in range(len(df)):
    demand = intercepts[i] - sensitivities[i] * prices[i]
    model.addConstr(demand <= capacities[i], name=f'capacity_{i}')

# Add constraints for price ordering within each product line and version
num_products_per_line = 3
for i in range(0, len(df), num_products_per_line):
    # Price ordering within each product line
    for j in range(num_products_per_line - 1):
        model.addConstr(prices[i+j+1] - prices[i+j] >= 0.01, name=f'price_order_{i+j}_{i+j+1}')
    # Price ordering between product lines for the same version
    if i > 0:
        model.addConstr(prices[i] - prices[i-num_products_per_line] >= 0.01, name=f'price_order_{i-num_products_per_line}_{i}')

# Optimize the model
model.optimize()

# Print the optimal prices and total revenue
if model.status == GRB.OPTIMAL:
    print("Optimal Prices:")
    for i in range(len(df)):
        print(f"{df.loc[i, 'Product']}: ${prices[i].X:.2f}")
    print(f"Total Revenue: ${model.ObjVal:.2f}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 17 rows, 9 columns and 25 nonzeros
Model fingerprint: 0xc2ce3ba1
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 5e+04]
Presolve removed 9 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 9 columns, 16 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 8.000e+00
 Factor NZ  : 3.600e+01
 Factor Ops : 2.040e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.34101576e+08  1.18618599e+09  5.00e+02 1.99e+02  

___